# Here is a test of the cubic grid search. I use PrO2 as an example.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import PyCrystalField as cef
import os
import scipy.io as sio
from functools import reduce
import time
from JensenTools import *

### Define the measured energy levels (from INS data) and define an allowable tolerance between calculated and measured energy.

In [12]:
tol = .05 #tolerance allowed between measured and calculated energy.
Emeas = [168, 335,385,730] #Boothroyd's measured levels of Sr2PrO4
comp = 'PrO2' #Compound name
gridDir = 'cubic_matrix_800x800/' #The directory of the saved 800x800 matrices

### In the following section we scan through all LS grids and find the (x,bpf) points that create matching energy levels.

In [13]:
print('Energies as measured by paper (meV):  ', Emeas)

LSNames, EList, data = loadMatrix(gridDir) #Load in all created 800x800 grids

for c in range(len(LSNames)):
    
    #Loading the x,bpf, and LS of each file.
    x = data[c]['X'][0]
    bpf = data[c]['B'][0]
    LS = data[c]['LS'][0][0]

#     plotContours(data[c],EList[c],Emeas,LSNames[c]) #Contour plotting for 4 E levels


    #Choose which bands to look for compatibilities.
    #For Boothroyd with 4 reported levels. I use indices 1-4.
    index = [1,2,3,4]
    Eindex = []
    EListindex = []
    for i in index:
        Eindex.append(Emeas[i-1])
        EListindex.append(EList[c][i-1])
        
    #Function call that searches for compatible (x,bpf) coordinates.
    coords = paramFinder(data[c],EListindex,Eindex,tol,comp,LSNames[c])

    #Printing results
    if len(coords) !=0:
        for j in [coords[0]]:
            print('!!! Compatibilities Found !!!')
            print('With x = ', x[j[0]], ' and bpf = ', bpf[j[1]])
            count = 1
            for i in EList[c]:
                print('E%i = '%count, data[c][i][j[0]][j[1]], 'meV')
                count += 1
            print()
    else:
        print('No compatibilities found')

    
    #If there is a compatibility then print an example of the matrix generated by PCF with cubic constraints.
    if(len(coords) != 0):
        print('\nFor ', LSNames[c])
        xind,bind = coords[0][0], coords[0][1]

        print('\nFor ', comp, ' at x[%i] = %.4f and bpf[%i] = %.4f'%(xind,x[xind],bind,bpf[bind]))
        print(EList[c][1], ' = ', data[c][EList[c][1]][xind][bind] )
        print('Using these values lets see if degeneracies are protected.\n')
        printPCFEigens(x[xind],bpf[bind],LS)


Energies as measured by paper (mev):   [168, 335, 385, 730]

Parameter search for:   Compound:  PrO2  at  LS = 60 meV with 0.050 tolerance.
No compatibilities found

Parameter search for:   Compound:  PrO2  at  LS = 70 meV with 0.050 tolerance.
No compatibilities found

Parameter search for:   Compound:  PrO2  at  LS = 100 meV with 0.050 tolerance.
!!! Compatibilities Found !!!
With x =  0.02127659574468077  and bpf =  -0.4067584480600751
E1 =  164.68384384074272 meV
E2 =  318.3330029128191 meV
E3 =  394.67335543818604 meV
E4 =  712.9069882304684 meV


For  LS = 100 meV

For  PrO2  at x[408] = 0.0213 and bpf[237] = -0.4068
E2  =  318.3330029128191
Using these values lets see if degeneracies are protected.


 Eigenvalues 	 Eigenvectors
		---------------------------------------------------------------------------------------------------------
0.00000 	|  [ 0.     0.     0.     0.281 -0.313  0.     0.     0.     0.     0.
  0.     0.281 -0.863  0.   ]  |
0.00000 	|  [ 0.101  0.     0.    